In [24]:
import numpy as np
import pandas as pd

from src.utils import generate_interactions_matrix, get_subsets_matrix_description
from src.evaluation.evaluation import downvote_seen_items, topn_recommendations, model_evaluate

from tqdm import tqdm
import matplotlib as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
training_yelp, testset_yelp, holdout_yelp, data_index_yelp, training_matrix_yelp, data_description_yelp = get_subsets_matrix_description('yelp',
                                                                                                                                         time_split_q=0.98)

training_movielens, testset_movielens, holdout_movielens, data_index_movielens, training_matrix_movielens, data_description_movielens = get_subsets_matrix_description('movielens',
                                                                                                                                         time_split_q=0.95)

print('Shapes, yelp data: ',training_yelp.shape[0], testset_yelp.shape[0], holdout_yelp.shape[0])
print('Shapes, movielens data: ',training_movielens.shape[0], testset_movielens.shape[0], holdout_movielens.shape[0])

Shapes, yelp data:  508436 197871 4688
Shapes, movielens data:  725370 221697 802


In [87]:
data_description_movielens

{'users': 'userid',
 'items': 'movieid',
 'feedback': 'rating',
 'n_users': 5227,
 'n_items': 3652}

In [71]:
# !!! scores_X - результат работы модели X, т.е. numpy.ndarray шейпа: scores_X.shape=(число юзеров в холдауте, число всех айтемов),
# где элементы - предсказанные скоры айтемов для каждого юзера (не индексы айтемов, а именно скоры).
scores_yelp = ...
scores_movielens = ...

In [ ]:
downvote_seen_items(scores_yelp, testset_yelp, data_description_yelp)
downvote_seen_items(scores_movielens, testset_movielens, data_description_movielens)

In [ ]:
topn = 10

recs_yelp = topn_recommendations(scores_yelp, topn=topn)
recs_movielens = topn_recommendations(scores_movielens, topn=topn)

In [72]:
datasets = ['Yelp: ', 'MovieLens: ', ]

In [ ]:
hr, mrr, ndcg, cov = model_evaluate(recs_yelp, holdout_yelp, data_description_yelp, topn=topn)
print(datasets[0])
print(f'HR@{topn} = {hr:.4f}')
print(f'MRR@{topn} = {mrr:.4f}')
print(f'nDCG@{topn} = {ndcg:.4f}')
print(f'COV@{topn} = {cov:.4f}')

In [ ]:
hr, mrr, ndcg, cov = model_evaluate(recs_movielens, holdout_movielens, data_description_movielens, topn=topn) 
print(datasets[1])
print(f'HR@{topn} = {hr:.4f}')
print(f'MRR@{topn} = {mrr:.4f}')
print(f'nDCG@{topn} = {ndcg:.4f}')
print(f'COV@{topn} = {cov:.4f}')

In [75]:
type(np.array([1,43,2]))

numpy.ndarray